In [1]:
# Import of the packages
import numpy as np
import scipy.optimize as optimize
import matplotlib.pyplot as plt
import math

In [93]:
# SVM classifier
class SVM:
    """ Support Vector Machine (SVM).
    
    Constructor to initialize the SVM with the different desired
    parameters and the type of SVM.
    
    Attributes:
        kerType: A string of the type of desired kernel.
        svmType: A string of the type of desired SVM.
        V: V-variable.
        C: C-variable.
        p: Degree of the polynomial kernel.
        sigma: Smoothing factor the Gaussian kernel.
        kappa: Scaling parameter of the sigmoid kernel.
        delta: Translation parameter of the sigmoid kernel.
        eps: Size of the insensitive tube.
    """

    EPSILON = 1e-5
    
    def __init__(self, kerType = 'linear', svmType = 'C',
                 V = 0.45, C = 1, p = 1, sigma = 1, 
                 kappa = 1, delta = 1, eps = 0.5):
        """ Initializes the SVM class."""
    
        
        self.kerType = kerType
        self.svmType = svmType
        self.V = V
        self.C = C
        self.p = p
        self.sigma = sigma
        self.kappa = kappa
        self.delta = delta
        self.eps = eps
    
    def kernel(self, x, y):
        """ Kernel computation.
        
        It computes the kernel value based on the dot product
        between two vectors.
        
        Args:
            x: Input vector.
            y: Other input vector.
            
        Returns:
            Kernel computation.
        """
        
        if self.kerType == "linear":
            k = np.dot(x,y) + 1
        elif self.kerType == "poly":
            k = (np.dot(x,y) + 1) ** self.p
        elif self.kerType == "radial":
            k = math.exp(-(np.dot(x-y,x-y))/(2*self.sigma))
        elif self.kerType == "sigmoid":
            k = math.atanh(self.kappa * np.dot(x,y) - self.delta)

        return k
    
    def lagrange(self, params, X, y, K):
        """
        TO DO
        """
        N2 = np.shape(params)[0]
        N = int(N2/2)
        a = params[0:N].reshape((N,1))
        a_hat = params[N:N2].reshape((N,1))
        y = y.reshape((N,1))
        if self.svmType == "C":
            L = (1/2) * (a - a_hat).T.dot(K).dot(a - a_hat) \
                + self.eps * np.sum(a + a_hat) \
                - (a - a_hat).T.dot(y)
            return L
    
    def train(self, X_tr, Y_tr):
        """TO DO"""
        N_tr = np.shape(X_tr)[0]
        K = np.zeros((N_tr, N_tr))
        for i in range(N_tr):
            for j in range(N_tr):
                K[i,j] = self.kernel(X_tr[i], X_tr[j])
        args = (X_tr, Y_tr, K)
        bounds = []
        for i in range(0, 2*N_tr):
            bounds.append((0,self.C))
        
        def constraint(params):
            N2 = np.shape(params)[0]
            N = int(N2/2)
            a = params[0:N]
            a_hat = params[N:N2]
            return np.sum(a - a_hat)
        
        cons = {'type':'eq', 'fun': constraint}
        initial = np.zeros(2*N_tr)
        final = optimize.minimize(self.lagrange, initial, args, \
            bounds=bounds, constraints=cons, method='SLSQP')
        
        if final.success != True:
            # exception shit
            return
        
        a = final.x[0:N_tr]
        a_hat = final.x[N_tr:2*N_tr]
        
        cond_valuable = abs((a - a_hat)) > 1e-10
        a = a[cond_valuable]
        a_hat = a_hat[cond_valuable]
        X_sv = X_tr[cond_valuable]
        Y_sv = Y_tr[cond_valuable]
        
        for i in range(np.shape(a)[0]):
            if (a[i] < self.C and a[i] > 0):
                yn = (Y_sv[i])
                ind = (i)
        

        b = yn - self.eps - (a - a_hat).T.dot(K)[ind[i]]
        b = np.mean(bs)
        
        self.a = a
        self.a_hat = a
        self.b = b
        self.X_sv = X_sv
        
    def pred(self, X):
        """ TO DO """
        
        N_sv = np.shape(self.X_sv)[0]
        N = np.shape(self.X)[0]
        a = self.a.reshape(N_sv,1)
        a_hat = self.a_hat.reshape(N_sv,1)
        K = np.zeros((N_sv, N))
        for i in range(N_sv):
            for j in range(N_ts):
                K[i,j] = self.kernel(X_sv[i],X[j])
        y = (a - a_hat).T.dot(K) + self.b
        y = y.reshape(-1)
        return y

In [94]:
N = 100
tr = 0.6
ts = 0.4
N_tr = int(N*tr)
N_ts = int(N*ts)
x = np.linspace(0,5,num=N)
y = 3 * x ** 3 + x + 2 + np.random.normal(loc=0, scale=1, size=N)
x_tr = x[0:N_tr]
y_tr = y[0:N_tr]
x_ts = x[N_ts:N]
y_ts = y[N_ts:N]

In [95]:
s = SVM(kerType='poly', C = 1, p = 4, eps = 0.31)
s.train(X_tr=x_tr, Y_tr=y_tr)

ValueError: shapes (49,) and (60,60) not aligned: 49 (dim 0) != 60 (dim 0)

In [11]:
X = np.ones((4))